# Assignment 4 - CS131
### Shashhank Seethula

### Task 1

In [12]:
from pyspark.sql import SparkSession

In [13]:
# Read the csv file stored in my apache_spark bucket
df = spark.read.csv("gs://apache_spark/2019-04.csv", header=True, inferSchema=True)

# Only select needed columns (passenger_count, pulocationid, dolocationid, total_amount)
df_new = df.select("passenger_count", "pulocationid", "dolocationid", "total_amount")

# Display the first 10 rows
df_new.show(10)

+---------------+------------+------------+------------+
|passenger_count|pulocationid|dolocationid|total_amount|
+---------------+------------+------------+------------+
|            1.0|       239.0|       239.0|         8.8|
|            1.0|       230.0|       100.0|         8.3|
|            1.0|        68.0|       127.0|       47.75|
|            1.0|        68.0|        68.0|         7.3|
|            1.0|        50.0|        42.0|       23.15|
|            1.0|        95.0|       196.0|         9.8|
|            1.0|       211.0|       211.0|         6.8|
|            1.0|       237.0|       162.0|         7.8|
|            1.0|       148.0|        37.0|        20.3|
|            1.0|       265.0|       265.0|        0.31|
+---------------+------------+------------+------------+
only showing top 10 rows



### Task 2

In [14]:
# Create an 80/20 split
trainDF, testDF = df_new.randomSplit([.8, .2], seed=42)

### Task 3

In [15]:
# Use VectorAssembler to create the features column
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(
    inputCols=["passenger_count", "pulocationid", "dolocationid"],
    outputCol="features")
vecTrainDF = vecAssembler.transform(trainDF)
vecTrainDF.select("passenger_count", "pulocationid", "dolocationid", "features", "total_amount").show(10)

[Stage 38:>                                                         (0 + 1) / 1]

+---------------+------------+------------+---------------+------------+
|passenger_count|pulocationid|dolocationid|       features|total_amount|
+---------------+------------+------------+---------------+------------+
|            0.0|         1.0|         1.0|  [0.0,1.0,1.0]|         0.3|
|            0.0|         1.0|         1.0|  [0.0,1.0,1.0]|        16.8|
|            0.0|         1.0|         1.0|  [0.0,1.0,1.0]|       114.3|
|            0.0|         1.0|       264.0|[0.0,1.0,264.0]|      119.75|
|            0.0|         3.0|       136.0|[0.0,3.0,136.0]|        22.8|
|            0.0|         4.0|         4.0|  [0.0,4.0,4.0]|        60.3|
|            0.0|         4.0|        49.0| [0.0,4.0,49.0]|        26.0|
|            0.0|         4.0|        68.0| [0.0,4.0,68.0]|       18.95|
|            0.0|         4.0|        75.0| [0.0,4.0,75.0]|       28.55|
|            0.0|         4.0|        79.0| [0.0,4.0,79.0]|         7.3|
+---------------+------------+------------+--------

In [16]:
# Use DecisionTreeRegressor to predict total_amount
from pyspark.ml.regression import DecisionTreeRegressor

dt = DecisionTreeRegressor(featuresCol="features", labelCol="total_amount")
dtModel = dt.fit(vecTrainDF)

### Task 4

In [17]:
# Create a pipeline with VectorAseember and DecisionTreeRegressor
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[vecAssembler, dt])

### Task 5

In [18]:
pipelineModel = pipeline.fit(trainDF)

### Task 6

In [19]:
# Retrieve predictions from model and show the top 10 rows
predDF = pipelineModel.transform(testDF)
predDF.select("passenger_count", "pulocationid", 
              "dolocationid", "features",
              "total_amount", "prediction").show(10)

[Stage 67:>                                                         (0 + 1) / 1]

+---------------+------------+------------+---------------+------------+------------------+
|passenger_count|pulocationid|dolocationid|       features|total_amount|        prediction|
+---------------+------------+------------+---------------+------------+------------------+
|            0.0|         1.0|         1.0|  [0.0,1.0,1.0]|       103.3|19.711640780100204|
|            0.0|         4.0|         4.0|  [0.0,4.0,4.0]|         6.8|19.711640780100204|
|            0.0|         4.0|        33.0| [0.0,4.0,33.0]|       31.55|15.648492562197445|
|            0.0|         4.0|        79.0| [0.0,4.0,79.0]|         7.8|15.648492562197445|
|            0.0|         4.0|       107.0|[0.0,4.0,107.0]|        11.8|15.648492562197445|
|            0.0|         4.0|       144.0|[0.0,4.0,144.0]|        11.3|17.806537849630722|
|            0.0|         4.0|       234.0|[0.0,4.0,234.0]|        11.0|17.806537849630722|
|            0.0|         7.0|       121.0|[0.0,7.0,121.0]|        28.8|17.80653

### Task 7

In [20]:
# Use RegressionEvaluator and display the RMSE between prediction and total_amount
from pyspark.ml.evaluation import RegressionEvaluator

regressionEvaluator = RegressionEvaluator(
    predictionCol="prediction",
    labelCol="total_amount",
    metricName="rmse")
rmse = regressionEvaluator.evaluate(predDF)

print("Root Mean Squared Error: " + str(rmse))

[Stage 68:==============>                                           (2 + 6) / 8]

Root Mean Squared Error: 12.206149635040326
